In [1]:
import json
import boto3
from botocore.exceptions import ClientError
import warnings
warnings.filterwarnings("ignore")

import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [2]:
def get_secret(secret_id, secret_key):
    region_name = "us-east-1"
    session = boto3.session.Session()
    client = session.client(service_name="secretsmanager", region_name=region_name)
    try:
        get_secret_value_response = client.get_secret_value(SecretId=secret_id)
    except ClientError as e:
        print(f"Failed to retrieve secret {secret_id}: {str(e)}")
        raise e
    secret = json.loads(get_secret_value_response["SecretString"])
    return secret[secret_key]

In [3]:
chat_api_key = get_secret("askjia-dev", "GPT35_API_KEY")
chat_api_endpoint = get_secret("askjia-dev", "GPT35_API_BASE")

embed_api_key = get_secret("askjia-dev", "EMBEDDING_API_KEY")
embed_api_endpoint = get_secret("askjia-dev", "EMBEDDING_API_BASE")

In [4]:
from langchain_openai import AzureOpenAIEmbeddings
from langchain_openai import AzureChatOpenAI

# Chat Completion
model = AzureChatOpenAI(azure_endpoint=chat_api_endpoint,
    api_key=chat_api_key,
    api_version="2023-05-15",
    azure_deployment="gpt-35-turbo-16k",
    temperature=0
)

# Embeddings
embeddings = AzureOpenAIEmbeddings(
    azure_endpoint=embed_api_endpoint,
    api_key=embed_api_key,
)

In [5]:
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings


In [6]:
## LOAD DOCS
loader = UnstructuredFileLoader(r"C:\\Projects\\Multi Agent\\Notebooks\\Data\\TV-POL-02234_v1.0.pdf")
docs = loader.load()

C:\Users\valmei14\AppData\Local\Temp\1\ipykernel_31208\3079032970.py:2: LangChainDeprecationWarning: The class `UnstructuredFileLoader` was deprecated in LangChain 0.2.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-unstructured package and should be used instead. To use it run `pip install -U :class:`~langchain-unstructured` and import as `from :class:`~langchain_unstructured import UnstructuredLoader``.
  loader = UnstructuredFileLoader(r"C:\\Projects\\Multi Agent\\Notebooks\\Data\\TV-POL-02234_v1.0.pdf")


In [7]:
## SPLIT DOCS
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

In [8]:
## VECTOR STORE
vectorstore = FAISS.from_documents(splits, embeddings)

In [9]:
## CREATE QA CHAIN
qa = RetrievalQA.from_chain_type(llm=model, chain_type="stuff", retriever=vectorstore.as_retriever())

In [10]:
query = "Fale sobre Judicialização?"
result = qa({"query": query})
print(result["result"])

C:\Users\valmei14\AppData\Local\Temp\1\ipykernel_31208\1772282440.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa({"query": query})


A judicialização refere-se ao fenômeno em que indivíduos ou grupos recorrem ao sistema judiciário com alegação de que os direitos à saúde, garantidos pela constituição ou por normas legais, não estão sendo respeitados. Isso pode ocorrer, por exemplo, quando há dificuldade de acesso a medicamentos que não estão sendo adequadamente fornecidos pelo sistema público de saúde ou pelos planos de saúde privados. Nesses casos, os pacientes podem optar por entrar com ações judiciais para garantir o acesso aos medicamentos necessários. A judicialização da saúde é um tema complexo e controverso, pois envolve questões legais, éticas e econômicas.


## Agents

#### QA Chain to StuffDocumentChain
- Nós podemos aprimorar essa chain, passando à ela um prompt especifico
- Aqui, estaremos usando o documento que foi carregado nas células passadas para simular um VectorStore.

In [11]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.vectorstores import InMemoryVectorStore
from langchain.agents import Tool, initialize_agent
from langchain.agents import AgentType
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain


In [12]:
# Usamos o documents carregados e splitados em 'split' para simular um banco vetorizado na memória, salvando no retriever.
vectorstore = InMemoryVectorStore.from_documents(
    documents=splits, embedding=embeddings
)
retriever = vectorstore.as_retriever()

In [13]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(model, prompt)

rag_chain = create_retrieval_chain(retriever, question_answer_chain)


In [14]:
response = rag_chain.invoke({"input": "Tipo de Acesso Formal?"})
response["answer"]

'O Tipo de Acesso Formal refere-se aos casos em que o paciente tem acesso aprovado aos medicamentos através de Pagadores (Operadora de Saúde ou SUS) ou quando o paciente faz o pagamento com recursos próprios. Nesses casos, o centro certificado pode optar por condições comerciais especiais ofertadas em contrato.'

### ReAct Agents

#### Primeiro Agent.
- Como primeiro agent, podemos criar um ReAct agent (Reasoning and Action).
- Um agent pode conter uma ou mais tools, nesse nosso caso, nosso agent vai possuir apenas a tool que realiza uma chamada para nossa rag_chain

In [15]:
from langchain.agents import create_react_agent

# Função que chama a chain rag
def rag_tool_func(input_text: str) -> str:
    response = rag_chain.invoke({"input": input_text})
    return response["answer"]

In [16]:
# Criamos uma tool capaz de chamar a função que utiliza o rag_chain.
rag_tool = Tool(
    name="Document RAG QA",
    func=rag_tool_func,
    description=(
        "Use este agente quando a pergunta for sobre informações presentes nos documentos carregados. "
        "Por exemplo, perguntas como 'O que significa Tipo de Acesso Formal?'."
    )
)

In [17]:

# Template para o agente RAG
rag_prompt_template = PromptTemplate(
    input_variables=["input"],
    template=(
        "Você é um agente que auxilia na resposta a perguntas com base em documentos fornecidos.\n"
        "Use as ferramentas disponíveis para obter informações adicionais.\n\n"
        "Pergunta: {input}\n"
    )
)

# Inicializar o agente RAG
rag_agent = initialize_agent(
    tools=[rag_tool],
    llm=model,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    agent_kwargs={"prompt": rag_prompt_template}
)


C:\Users\valmei14\AppData\Local\Temp\1\ipykernel_31208\2337228908.py:12: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 1.0. Use :meth:`~Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc.` instead.
  rag_agent = initialize_agent(


In [18]:
#Fazer uma pergunta baseada nos documentos
document_response = rag_agent.run("O que é 'Tipo de Acesso Formal'?")
print(document_response)

C:\Users\valmei14\AppData\Local\Temp\1\ipykernel_31208\3395987160.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  document_response = rag_agent.run("O que é 'Tipo de Acesso Formal'?")




> Entering new AgentExecutor chain...
I should use the Document RAG QA tool to find the answer in the documents.
Action: Document RAG QA
Action Input: "O que é 'Tipo de Acesso Formal'?"
Observation: O Tipo de Acesso Formal refere-se aos casos em que o paciente tem acesso aprovado aos medicamentos por meio de pagadores, como operadoras de saúde ou o Sistema Único de Saúde (SUS), ou quando o paciente faz o pagamento com recursos próprios. Nesses casos, o centro certificado pode optar por condições comerciais especiais oferecidas em contrato.
Thought:I now know the final answer
Final Answer: O Tipo de Acesso Formal refere-se aos casos em que o paciente tem acesso aprovado aos medicamentos por meio de pagadores, como operadoras de saúde ou o Sistema Único de Saúde (SUS), ou quando o paciente faz o pagamento com recursos próprios. Nesses casos, o centro certificado pode optar por condições comerciais especiais oferecidas em contrato.

> Finished chain.
O Tipo de Acesso Formal refere-se ao

### =====================================================================================

##### Chamadas diretas
- Para chamadas que não necessáriamente precisam de conteúdos pré carregados para serem respondidos, podemos usar uma LLMChain usando o modelo direto, com um simples prompt formato

In [19]:
# Template para o agente do modelo
model_prompt_template = PromptTemplate(
    input_variables=["input"],
    template=(
        "Você é um modelo treinado para responder a perguntas com base no conhecimento adquirido durante seu treinamento.\n"
        "Responda de forma clara e objetiva.\n\n"
        "Pergunta: {input}\n"
    )
)

# Cadeia direta com o modelo e o prompt
model_chain = LLMChain(
    llm=model,
    prompt=model_prompt_template
)

C:\Users\valmei14\AppData\Local\Temp\1\ipykernel_31208\905444249.py:12: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  model_chain = LLMChain(


In [20]:
# Fazer uma pergunta
response = model_chain.invoke("Explique o conceito de aprendizado por reforço.")
print(response)

{'input': 'Explique o conceito de aprendizado por reforço.', 'text': 'O aprendizado por reforço é um conceito na área de inteligência artificial e aprendizado de máquina que se baseia em um agente aprender a tomar decisões através de interações com um ambiente. Nesse tipo de aprendizado, o agente recebe feedback em forma de recompensas ou punições, dependendo das ações que ele toma. O objetivo é maximizar as recompensas ao longo do tempo, aprendendo a tomar as melhores decisões em diferentes situações. O agente utiliza técnicas de exploração e exploração para descobrir a melhor estratégia de ação, e o aprendizado ocorre através de tentativa e erro. O aprendizado por reforço é amplamente utilizado em áreas como jogos, robótica e otimização de processos.'}


In [21]:
# Podemos transformar esse LLMChain ccomo uma tool, encapsulando a chamada .run em uma função model_tool_func

def model_tool_func(input_text: str) -> str:
    return model_chain.run(input_text)

# Primeiro, devemos criar uma tool com a chain. Será nossa model_tool. A tool que responde com informações do treinamento.
model_tool = Tool(
    name="Model Agent",
    func=model_tool_func,
    description=(
        "Use este agente quando a pergunta for sobre conhecimento geral, conceitos ou fatos "
        "aprendidos pelo modelo, como 'O que é aprendizado por reforço?'."
    )
)

# Assim, construimos uma tool que responde ao usuário sem se basear em documentos pré carregados

### =====================================================================================

##### Agent Supervisor
- Como sabemos, podemos criar agents com uma ou mais tools.
- Cada Tool possui sua descrição, portanto o agent terá conhecimento dos poderes das tools que tem em mãos.
- Portanto, pdemos criar um supervisor_agent que possui as duas tools.
- Assim, sempre que uma pergunta for realizada para esse agent, ele decidirá quem vai responder a pergunta. Se é a rag_tool ou o model_tool.

In [22]:
# Agora criamos o agent_supervisor. Ele terá conhecimento das duas tools e será instruido via prompt a responder a pergunta usando a tool correta
supervisor_prompt_template = PromptTemplate(
    input_variables=["input"],
    template=(
        "Você é um supervisor que decide qual agente usar para responder a uma pergunta.\n"
        "Se a pergunta for sobre informações nos documentos carregados, use o agente RAG.\n"
        "Se for uma pergunta geral ou conceitual, use o agente Model.\n\n"
        "Pergunta: {input}\n"
    )
)


# Inicializar o agente supervisor
supervisor_agent = initialize_agent(
    tools=[rag_tool, model_tool],
    llm=model,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    agent_kwargs={"prompt": supervisor_prompt_template}
)


In [23]:
# Agora sempre que uma pergunta for realizada, ele vai verificar se consegue responder usando a rag_tool, caso não consiga, vai responder usando a model_tool.

response1 = supervisor_agent.run("Fluxo de Price Override para Carvykti?")
print(response1)

# response2 = supervisor_agent.run("Explique o conceito de aprendizado por reforço.")
# print("Resposta do Supervisor (RAG ou Model):", response2)



> Entering new AgentExecutor chain...
I'm not familiar with the term "Carvykti". I should use the Document RAG QA tool to find information about "Fluxo de Price Override".
Action: Document RAG QA
Action Input: "Fluxo de Price Override"
Observation: O fluxo de Price Override é um processo que permite a sobreposição ou ajuste manual de preços quando há uma divergência entre o preço cadastrado e o preço negociado para uma transação. Esse processo é utilizado em situações pré-determinadas em que o preço padrão não atende às necessidades específicas da ordem de venda. O time de pricing é responsável por analisar e aprovar as condições comerciais a serem aplicadas em cada Sales Order, de acordo com o contrato vigente, política comercial e tipo de acesso do paciente.
Thought:I now know the final answer
Final Answer: O fluxo de Price Override é um processo que permite a sobreposição ou ajuste manual de preços quando há uma divergência entre o preço cadastrado e o preço negociado para uma tra

### =====================================================================================

## Langgraph

##### Como utilizar nossas chains com o langgraph

In [24]:
from typing import Sequence

from langchain_core.messages import AIMessage, BaseMessage, HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, StateGraph, END
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict

In [25]:
# Ainda estamos utilizando a mesma chain criada inicialmente com o retriever. Dessa vez, utilizando a variável {chat_history} que será implementada.
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "Chat History:\n{chat_history}\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(model, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [26]:
# Definimos um dict representando o estado da aplicação. Esse dict é chamado de State. O state armazena o valor das variáveis que podem/devem ser persistida entre as interações.
class State(TypedDict):
    input: str
    chat_history: Annotated[Sequence[BaseMessage], add_messages]
    context: str
    answer: str

In [27]:
# Definimos então um nó simples que executa o `rag_chain`.
# Os valores `return` do nó atualizam o estado do gráfico, então aqui nós apenas atualizamos o histórico de bate-papo com a mensagem de entrada e a resposta.
def call_rag_chain(state: State):
    response = rag_chain.invoke(state)
    return {
        "chat_history": [
            HumanMessage(state["input"]),
            AIMessage(response["answer"]),
        ],
        "context": response["context"],
        "answer": response["answer"],
    }

In [28]:
# Instanciando o fluxo de trabalho
workflow = StateGraph(state_schema=State)

In [29]:
# Adicionamos o ponto de partida e nosso node.
workflow.add_edge(START, "rag")
# Mapeamos que a saida "rag" vai executar o node call_rag_chain.
workflow.add_node("rag", call_rag_chain)

In [30]:
# Finalmente, compilamos o gráfico com um objeto checkpointer.
# Isso persiste o estado, neste caso na memória.
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [31]:

config = {"configurable": {"thread_id": "1"}}

In [32]:
result = app.invoke(
    {"input": "O que é cancelamento?"},
    config=config,
)
print(result["answer"])

Cancelamento é o ato de desistir de uma ordem de venda ou de uma medicação pelo centro certificado. Pode ser solicitado pelo centro certificado através do sistema CCM/CQUENCE e pode ocorrer antes da infusão do produto no paciente. O cancelamento pode ser motivado por razões clínicas, como o óbito do paciente, ou por razões não clínicas, como a desistência do paciente.


In [33]:
result = app.invoke(
    {"input": "Fale mais sobre os motivos?"},
    config=config,
)
print(result["answer"])

Os motivos clínicos para o cancelamento podem incluir o óbito do paciente ou razões relacionadas à saúde do paciente. Já os motivos não clínicos podem envolver a desistência do paciente. Esses motivos devem ser documentados pelo centro certificado e, caso solicitado pela J&J, o centro deve apresentar a documentação comprobatória do cancelamento.


In [34]:
chat_history = app.get_state(config).values["chat_history"]
for message in chat_history:
    message.pretty_print()

================================ Human Message =================================

O que é cancelamento?
================================== Ai Message ==================================

Cancelamento é o ato de desistir de uma ordem de venda ou de uma medicação pelo centro certificado. Pode ser solicitado pelo centro certificado através do sistema CCM/CQUENCE e pode ocorrer antes da infusão do produto no paciente. O cancelamento pode ser motivado por razões clínicas, como o óbito do paciente, ou por razões não clínicas, como a desistência do paciente.
================================ Human Message =================================

Fale mais sobre os motivos?
================================== Ai Message ==================================

Os motivos clínicos para o cancelamento podem incluir o óbito do paciente ou razões relacionadas à saúde do paciente. Já os motivos não clínicos podem envolver a desistência do paciente. Esses motivos devem ser documentados pelo centro certificado e, 

#### Construindo o Graph com o model chain:

In [35]:
def call_model_chain(state: State):
    response = model_chain.invoke(state)
    return {
        "chat_history": [
            HumanMessage(state["input"]),
            AIMessage(response["text"]),
        ],
        "answer": response["text"],
    }

In [36]:
# Instanciando o fluxo de trabalho
workflow = StateGraph(state_schema=State)

In [37]:
# Adicionamos o ponto de partida e nosso node.
workflow.add_edge(START, "model")
workflow.add_node("model", call_model_chain)

In [38]:
# Finalmente, compilamos o gráfico com um objeto checkpointer.
# Isso persiste o estado, neste caso na memória.
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [39]:
config = {"configurable": {"thread_id": "1"}}

In [40]:
result = app.invoke(
    {"input": "Quem foi michael jackson?"},
    config=config,
)
result['answer']

'Michael Jackson foi um cantor, compositor e dançarino norte-americano. Ele é considerado um dos artistas mais influentes e icônicos da história da música pop. Jackson alcançou grande sucesso com álbuns como "Thriller" e "Bad", e é conhecido por hits como "Billie Jean", "Beat It" e "Smooth Criminal". Ele também foi um inovador em termos de coreografias e videoclipes. Infelizmente, Michael Jackson faleceu em 2009, deixando um legado duradouro na indústria musical.'

#### Supervisor Node

In [41]:
# Ainda estamos utilizando a mesma chain criada inicialmente com o retriever. Dessa vez, utilizando a variável {chat_history} que será implementada.
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "Chat History:\n{chat_history}\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(model, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)


In [42]:
# Fazemos o mesmo para a chain que utiliza informações externas, devemos adicionar o histórico.
model_prompt_template = PromptTemplate(
    input_variables=["input", "chat_history"],
    template=(
        "Você é um modelo treinado para responder a perguntas com base no conhecimento adquirido durante seu treinamento.\n"
        "Responda de forma clara e objetiva.\n\n"
        "Utilize o histórico de conversas do usuário para uma precisão melhor nas respostas: {chat_history}"
        "Pergunta: {input}\n"
    )
)

# Cadeia direta com o modelo e o prompt
model_chain = LLMChain(
    llm=model,
    prompt=model_prompt_template
)

In [43]:
# Criamos novamente nosso state, dessa vez adiccionando mais uma variável 'next'
# Essa variável vai armazenar o valor da proxima ação do fluxo seguindo o supervisor.

class State(TypedDict):
    input: str
    chat_history: Annotated[Sequence[BaseMessage], add_messages]
    context: str
    answer: str
    next: str

In [44]:
# O supervisor é um nó central no fluxo do LangGraph.
# Seu papel é analisar a entrada do usuário e o histórico de mensagens
# para decidir qual próximo nó no workflow deve ser acionado.
# Ele pode escolher entre:
# - "model": Para perguntas genéricas que não dependem de informações específicas.
# - "rag": Para perguntas relacionadas a um conjunto de documentos específicos (neste caso, sobre Carvykti).
# A decisão do supervisor é baseada no prompt que combina o input atual do usuário e o histórico do chat.

def supervisor(state):
    # Obtém o input atual do usuário e o histórico de mensagens do estado.
    question = state['input']
    messages = state['chat_history']
    prompt = f"""
    Você é um agent decisivo em um workflow langgraph.
    Seu papel é analisar o input do usuário e definir qual a proxima etapa do fluxo.
    Suas opções são:
        - model: Responder com o modelo padrão, para respostas genéricas
        - rag: Responder com a chain de rag, para respostas relacionadas ao arquivo com informações a respeito da Comercialização de Carvykti.
    Sua resposta deve ser APENAS a opção que deve ser usada pelo workflow.
    Analise o histórico de mensagens para melhor precisão em sua escolha:
    {messages}
    Input do usuário:
    {question}
    Qual node devemos usar, model ou rag?
    """
    # O modelo é chamado com o prompt para decidir a próxima etapa.
    supervisor_answer_raw = model.invoke([{"role": "system", "content": prompt}])

    # O modelo retorna sua decisão ("model" ou "rag").
    supervisor_answer = supervisor_answer_raw.content.strip()
    print(f"Supervisor escolheu usar:\n{supervisor_answer}")
    
    # Atualiza o estado com a decisão tomada.
    state.update({"next": supervisor_answer})
    return state

In [45]:
# A função `validate_subject` é usada para validar a decisão tomada pelo supervisor.
# Essa função verifica o valor da chave `next` no estado, que foi atualizado pelo supervisor.
# Com base no valor de `next`, ela retorna o nome do próximo nó no workflow.
# Se `next` não for válido, a função lança um erro.

def validate_subject(state):
    try:
        # Obtém o próximo nó definido pelo supervisor a partir do estado.
        next = state['next']
        
        # Se o próximo nó for "model", retorna o nome do nó "model".
        if next == "model":
            return "model"
        # Se o próximo nó for "rag", retorna o nome do nó "rag".
        elif next == "rag":
            return "rag"
    except Exception as e:
        # Caso ocorra algum erro (ex.: a chave `next` não existe no estado),
        # a função lança uma exceção com uma mensagem explicativa.
        raise RuntimeError("Erro ao validar o subject")

In [46]:
# A função `call_model_chain` é responsável por executar o nó que utiliza a chain padrão do modelo.
# Ela invoca o `model_chain`, que foi configurado para lidar com perguntas genéricas.
# Essa função também atualiza o histórico de mensagens (`chat_history`) no estado, adicionando
# a entrada do usuário e a resposta do modelo.

def call_model_chain(state: State):
    # Invoca a `model_chain` com o estado atual, que contém o input do usuário.
    response = model_chain.invoke(state)
    
    # Retorna um novo estado atualizado, incluindo:
    # - O histórico de mensagens atualizado com a entrada do usuário (`HumanMessage`) e a resposta do modelo (`AIMessage`).
    # - A resposta gerada pelo modelo no campo `answer`.
    return {
        "chat_history": [
            HumanMessage(state["input"]),
            AIMessage(response["text"]), 
        ],
        "answer": response["text"],  
    }


In [47]:
# A função `call_rag_chain` é responsável por executar o nó que utiliza RAG.
# Ela invoca a `rag_chain`, que combina recuperação de documentos com geração de respostas.
# Essa função também atualiza o histórico de mensagens (`chat_history`) no estado,
# além de incluir o contexto retornado pela recuperação.

def call_rag_chain(state: State):
    # Invoca a `rag_chain` com o estado atual, que contém o input do usuário e possivelmente outras variáveis.
    response = rag_chain.invoke(state)

    # Retorna um novo estado atualizado, incluindo:
    # - O histórico de mensagens atualizado com a entrada do usuário (`HumanMessage`) e a resposta do modelo (`AIMessage`).
    # - O contexto recuperado pela RAG no campo `context`.
    # - A resposta gerada pela RAG no campo `answer`.
    return {
        "chat_history": [
            HumanMessage(state["input"]),  
            AIMessage(response["answer"]),  
        ],
        "context": response["context"],
        "answer": response["answer"],
    }


In [48]:
# Instanciando o fluxo de trabalho
workflow = StateGraph(state_schema=State)


In [49]:
# Adicionamos o ponto de partida e o nó supervisor ao workflow.
# O nó "supervisor" é responsável por analisar o input do usuário e decidir o próximo passo no fluxo.
workflow.add_edge(START, "supervisor")
workflow.add_node("supervisor", supervisor)

In [50]:
# Configuramos arestas condicionais para o nó "supervisor".
# O supervisor atualiza o estado com a chave `next`, que pode ser "rag" ou "model".
# Com base no valor de `next`, o workflow segue para o nó correspondente.
workflow.add_conditional_edges("supervisor", validate_subject, {
    "rag": "rag",      
    "model": "model",
})

In [51]:
# Adicionamos os nós que representam as chains:
workflow.add_node("rag", call_rag_chain)
workflow.add_node("model", call_model_chain)

In [52]:
# Configuramos o checkpoint para persistir o estado do workflow.
# Aqui utilizamos o `MemorySaver`, que armazena o estado em memória.
memory = MemorySaver()

# Compilamos o workflow para criar o objeto `app`.
# Esse objeto pode ser usado para invocar o fluxo com inputs específicos.
app = workflow.compile(checkpointer=memory)

In [53]:
# "configurable" a chave principal que agrupa todas as configurações personalizáveis para o workflow.
# "thread_id" representa um identificador único para a execução atual do workflow

config = {"configurable": {"thread_id": "2"}}

In [54]:
result = app.invoke(
    {"input": "Quem foi Michael Jackson?"},
    config=config,
)
result['answer']

Supervisor escolheu usar:
model


'Resposta: Michael Jackson foi um cantor, compositor e dançarino norte-americano. Ele é considerado um dos artistas mais influentes da história da música pop. Jackson alcançou grande sucesso com álbuns como "Thriller" e "Bad", e é conhecido por hits como "Billie Jean" e "Beat It". Ele também foi um dos pioneiros dos videoclipes musicais. Além de sua carreira musical, Jackson também foi um filantropo e ativista pelos direitos humanos. Ele faleceu em 2009.'

In [55]:
result = app.invoke(
    {"input": "Quais seus maiores sucessos?"},
    config=config,
)
result['answer']

Supervisor escolheu usar:
model


'Os maiores sucessos de Michael Jackson incluem músicas como "Thriller", "Billie Jean", "Beat It", "Bad", entre outros. Ele também alcançou grande sucesso com os álbuns "Thriller" e "Bad".'

In [56]:
chat_history = app.get_state(config).values["chat_history"]
for message in chat_history:
    message.pretty_print()

================================ Human Message =================================

Quem foi Michael Jackson?
================================== Ai Message ==================================

Resposta: Michael Jackson foi um cantor, compositor e dançarino norte-americano. Ele é considerado um dos artistas mais influentes da história da música pop. Jackson alcançou grande sucesso com álbuns como "Thriller" e "Bad", e é conhecido por hits como "Billie Jean" e "Beat It". Ele também foi um dos pioneiros dos videoclipes musicais. Além de sua carreira musical, Jackson também foi um filantropo e ativista pelos direitos humanos. Ele faleceu em 2009.
================================ Human Message =================================

Quais seus maiores sucessos?
================================== Ai Message ==================================

Os maiores sucessos de Michael Jackson incluem músicas como "Thriller", "Billie Jean", "Beat It", "Bad", entre outros. Ele também alcançou grande sucesso com o